In [1]:
import numpy as np
import csv
import pandas as pd
import os
import time
from tqdm import tqdm
from multiprocessing import Pool

In [2]:
df_ques = pd.read_pickle('/Users/s0c02nj/Downloads/df_query_ms.pkl')

In [3]:
df_res = pd.read_pickle('/Users/s0c02nj/Downloads/df_result_ms.pkl')

In [4]:
df_ques

,qid,pid,query,passage
0,188714,1000052,foods and supplements to lower blood sugar,Watch portion sizes: ■ Even healthy foods will...
1,1082792,1000084,what does the golgi apparatus do to the protei...,"Start studying Bonding, Carbs, Proteins, Lipid..."
2,995526,1000094,where is the federal penitentiary in ind,It takes THOUSANDS of Macy's associates to bri...
3,199776,1000115,health benefits of eating vegetarian,The good news is that you will discover what g...
4,660957,1000115,what foods are good if you have gout?,The good news is that you will discover what g...
...,...,...,...,...
6667572,959083,984571,when was niagara falls created,Quick Search for Gas Prices Find the lowest ga...
6667592,199776,984719,health benefits of eating vegetarian,Hemp is an excellent protein source for vegeta...
6668346,1099065,993530,how far deep to plant beet early wonder,"Dipladenia Rio Deep Red, Dipladenia 'Rio™ Deep..."
6668574,1084910,995866,what disease do roof rats cause,"Roof Shapes, Overhangs and Materials Gables, h..."


In [5]:
df_res.head()

,qid,random1,pid,random2
492,1084910,0,7123877,1
666,1082792,0,4879323,1
1439,1019649,0,7234626,1
1939,1099065,0,7290134,1
2359,167436,0,7352415,1


In [6]:
from elasticsearch import helpers, Elasticsearch
import csv
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from inflection import singularize
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine as cs

In [7]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.info()

{'name': 'm-c02wv1jnhtd5',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'i0Q_KCKeTLGlRZzF8qt__A',
 'version': {'number': '7.6.2',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'ef48eb35cf30adf4db14086e8aabd07ef6fb113f',
  'build_date': '2020-03-26T06:34:37.794943Z',
  'build_snapshot': False,
  'lucene_version': '8.4.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [8]:
def text_preprocessing(text):
    text= text.lower()
    text= re.sub(r'[^a-z]',' ',text)
    #text= " ".join([s for s in text.split() if len(s)>2])
    #text= " ".join([x for x in text.split() if x not in stopwords.words('english')])
    #text= " ".join([inflection.singularize(x) for x in text.split()])
    text= text.strip()
    return(text)

In [9]:
tqdm.pandas()

In [10]:
df_ques['passage'] = df_ques['passage'].progress_apply(lambda x: text_preprocessing(x))

df_ques['query'] = df_ques['query'].progress_apply(lambda x: text_preprocessing(x))

100%|██████████| 20000/20000 [00:00<00:00, 142523.06it/s]


In [11]:
index='index_v1'

In [12]:
def doc_generator(data,index):
    df_iter= data.iterrows()
    
    for index,document in df_iter:
        yield{
            "_index": 'index_v1',
            "_type": "_doc",
            "_id" : f"{index}",
            "_source": document.to_json()
        }
    raise StopIteration

In [13]:
df_ques['pid']

0          1000052
1          1000084
2          1000094
3          1000115
4          1000115
            ...   
6667572     984571
6667592     984719
6668346     993530
6668574     995866
6668591     996086
Name: pid, Length: 20000, dtype: int64

In [14]:
for success,info in tqdm(helpers.parallel_bulk(es,doc_generator(df_ques[['passage']],index))):
    if not success:
        print('A document failed:', info)
es.indices.refresh(index=index)

20000it [00:03, 5022.29it/s]


{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

In [67]:
def SEARCH(text,index,field):
    
    res= es.search(index=index,body={"query":{"match":{field:{"query":text,"operator":"or","fuzziness": "0"
                                                            }}}},size = 100)
    
    return([(x.get('_source'),x.get('_score')) for x in res['hits']['hits']])

In [68]:
index='index_v1'
field = 'passage'

In [69]:
#SEARCH('corporation',index,field)
question_un = list(df_ques['query'].unique())

df_new = pd.DataFrame()
df_new['question'] = question_un

In [70]:
def query_generator(df):
    while True:
        yield (df_new['question'])

In [71]:
query_generator(df_new[['question']])

<generator object query_generator at 0x108b7aeb8>

In [72]:
import concurrent.futures

In [73]:
%%time

queries=[]
with concurrent.futures.ThreadPoolExecutor() as executor:
    queries={executor.submit(SEARCH, query,index,'passage'): query for query in tqdm(df_new.question)}


100%|██████████| 20/20 [00:00<00:00, 488.69it/s]

CPU times: user 30.5 ms, sys: 10.2 ms, total: 40.7 ms
Wall time: 54.9 ms


In [74]:
out = [x.result() for x in tqdm(queries)]

100%|██████████| 20/20 [00:00<00:00, 68255.56it/s]


In [75]:
len(out)
out[0][1]

({'passage': 'in people with type   diabetes  supplements made from one type of broiled prickly pear cactus stem may lower blood sugar levels  early research shows that the supplements may lower blood sugar by     to     in some people only one species    opuntia streptacantha    had this effect n people with type   diabetes  supplements made from one type of broiled prickly pear cactus stem may lower blood sugar levels  early research shows that the supplements may lower blood sugar by     to     in some people'},
 20.598402)

In [76]:
def get_std_ref_match(data_current):
    
    #UPC to match
    questions = []
    
    #Original
    answers = []
    
    #Score
    score_bm25 = []
    
    #qid
    #qs_id = []
    
    #pid
    #ps_id = []
    
        
    
    #Reindex
    data_current.index = range(0,len(data_current))
    
    
    for i in tqdm(range(0,len(data_current))):

            #indexing
            #data_current.index = range(0,len(data_current))
            
            #question
            qs = data_current['question'].iloc[i]
            
           
            #Getting the search result
            temp_out = out[i]
            val = len(temp_out)

            if val ==0:
                
                #questions
                questions.append(qs)
                

                #answer
                answers.append('NA')
                
                #qs_id ps_9d
                #qs_id.append('NA')
                #ps_id.append('NA')
                
                #Score
                score_bm25.append('NA')
                
             
            else:
                for j in np.arange(val):

                    #clean_ingred
                    questions.append(qs)
                

                    #Creating temp
                    temp2 = temp_out[j]

                    #Separting the tuple
                    doc_txt = temp2[0]['passage']
                    #qs_i = temp2[0]['qid']
                    #ps_i = temp2[0]['pid']
                    
                    #query,passage ids
                    #qs_id.append(qs_i)
                    #ps_id.append(ps_i)

                    #bm25
                    doc_scr = temp2[1]


                    #matched answers
                    answers.append(doc_txt)
                    

                    #Score
                    score_bm25.append(doc_scr)
                    
                    
                    
    

    #Converting to a Dataframe   
    df_out = pd.DataFrame()
    
    #df_out
    #df_out['pid'] = ps_id
    df_out['questions'] = questions
    df_out['passage'] = answers
    df_out['score'] = score_bm25

    return df_out



In [77]:
df_out = get_std_ref_match(df_new)

100%|██████████| 20/20 [00:00<00:00, 5334.91it/s]


In [78]:
df_out.shape

(2000, 3)

In [79]:
df_ques_id = df_ques[['qid','query']]

df_ques_id = df_ques_id.drop_duplicates()

In [80]:
df_pass_id = df_ques[['pid','passage']]

df_pass_id = df_pass_id.drop_duplicates()

In [81]:
df_ques_id.shape

(20, 2)

In [82]:
df_pass_id.head()

,pid,passage
0,1000052,watch portion sizes even healthy foods will...
1,1000084,start studying bonding carbs proteins lipid...
2,1000094,it takes thousands of macy s associates to bri...
3,1000115,the good news is that you will discover what g...
5,1000130,the pancreas secretes pancreatic enzyme which ...


In [83]:
df_out1 = pd.merge(left= df_out ,
                      right= df_ques_id,
                      left_on= 'questions',
                      right_on = 'query',
                      how='left')

In [84]:
df_out1 = df_out1.drop(['query'],axis=1)

In [85]:
df_out2 = pd.merge(left= df_out1 ,
                      right= df_pass_id,
                      on= 'passage',
                      how='left')

In [86]:
df_out2[['questions','passage','qid','pid']].to_pickle('/Users/s0c02nj/Downloads/df_ques_bm25.pkl')

In [87]:
df_out2

,questions,passage,score,qid,pid
0,foods and supplements to lower blood sugar,easy tips to reduce insulin and lose belly fat...,20.708996,188714,3573129
1,foods and supplements to lower blood sugar,in people with type diabetes supplements ma...,20.598402,188714,4865944
2,foods and supplements to lower blood sugar,ohio state university researchers saw insulin...,20.147104,188714,4321742
3,foods and supplements to lower blood sugar,folate may cause low blood pressure caution i...,20.016598,188714,7469016
4,foods and supplements to lower blood sugar,foods that lower blood sugar for people with d...,19.844475,188714,5439925
...,...,...,...,...,...
2001,does ichigo lose his powers,at the same time the number on ichigo s be...,10.695086,167436,644061
2002,does ichigo lose his powers,superman s powers and abilities the powers of...,10.660635,167436,3792806
2003,does ichigo lose his powers,ichigo kurosaki kurosaki ichigo is a ...,10.643302,167436,2939839
2004,does ichigo lose his powers,in the third austin powers film goldmember d...,10.588695,167436,7822002


In [34]:
df_pred = df_out2.sort_values(by=['qid','score'],ascending = False)

In [35]:
df_pred['indices'] = range(0,len(df_pred))
df_pred['rank'] = (df_pred.groupby('qid')['indices'].rank(ascending=True))
df_pred = df_pred.drop(['indices'],axis=1)

In [36]:
df_final = df_pred.groupby('qid').head(10).reset_index(drop=True)

In [37]:
df_final_v1 = df_final[['qid','pid','rank']]

In [42]:
MaxMRRRank = 10
def compute_metrics(qids_to_relevant_passageids, qids_to_ranked_candidate_passages):
    """Compute MRR metric
    Args:    
    p_qids_to_relevant_passageids (dict): dictionary of query-passage mapping
        Dict as read in with load_reference or load_reference_from_stream
    p_qids_to_ranked_candidate_passages (dict): dictionary of query-passage candidates
    Returns:
        dict: dictionary of metrics {'MRR': <MRR Score>}
    """
    all_scores = {}
    MRR = 0
    qids_with_relevant_passages = 0
    ranking = []
    list1=list(set(qids_to_ranked_candidate_passages['qid'].to_list()))
    #print(list1)
    list2=qids_to_relevant_passageids['qid'].to_list()
    for qid in list1:
        if qid in list2:
            ranking.append(0)
            target_pid = list(qids_to_relevant_passageids[qids_to_relevant_passageids['qid']==qid]['pid'])
            candidate_pid = list(qids_to_ranked_candidate_passages[qids_to_ranked_candidate_passages['qid']==qid]['pid'])
            for i in range(0,MaxMRRRank):
                if candidate_pid[i] in target_pid:
                    MRR += 1/(i + 1)
                    #print(MRR)
#                     ranking.pop()
#                     ranking.append(i+1)
                    break
#     if len(ranking) == 0:
#         raise IOError("No matching QIDs found. Are you sure you are scoring the evaluation set?")
# #     print(MRR)
#     print(len(qids_to_relevant_passageids))
    MRR = MRR/len(list1)
    all_scores['MRR @10'] = MRR
    all_scores['QueriesRanked'] = qids_to_ranked_candidate_passages['qid'].nunique()
    return all_scores

In [43]:
mrr = compute_metrics(df_res, df_final_v1)

In [44]:
mrr

{'MRR @10': 0.09791666666666667, 'QueriesRanked': 20}